In [1]:
import seaborn as sns
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np

from sklearn import tree
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score, mean_squared_error
from sklearn.feature_selection import SelectFromModel
from dtreeviz.trees import dtreeviz

ImportError: cannot import name 'dtreeviz' from 'dtreeviz.trees' (c:\Users\chobi\anaconda3\envs\myenv\lib\site-packages\dtreeviz\trees.py)

In [2]:
raw_data = pd.read_csv('train.csv')
y = pd.DataFrame(raw_data['SalePrice'].copy(), columns=['SalePrice'])
raw_data = raw_data.drop('SalePrice', axis = 1)
y =np.ravel(y)



In [3]:
y

array([208500, 181500, 223500, ..., 266500, 142125, 147500], dtype=int64)

In [4]:
num_data_columns = ['GrLivArea', 'TotalBsmtSF', '1stFlrSF', 'YearBuilt', 'YearRemodAdd','GarageArea', 'LotArea', 'BsmtFinSF1', 'TotRmsAbvGrd']
label_data_columns = ['OverallQual','FullBath', 'ExterQual', 'KitchenQual', 'OverallCond']
one_hot_columns = ['GarageCars', 'Neighborhood', 'GarageType', 'Fireplaces']

In [5]:
num_data = raw_data[num_data_columns]
label_data = raw_data[label_data_columns].astype('category') #because we about to work with LabelEncoder so the requirement of category type is in needed
one_hot_data = raw_data[one_hot_columns]

new_data = pd.concat([num_data,label_data,one_hot_data], axis = 1)

In [6]:
def show_missing(data):
    #returning pandas dataframes describing about missing datas in columns

    variables = []
    dtypes = []
    count = []
    missing = []
    #check if there are any missing value here?
    missing_vale_count = data.isnull().sum()
    missing_column = missing_vale_count[missing_vale_count>0]
    
    #print out the detail of data
    for r in data[missing_column.index]:
        variables.append(r)
        dtypes.append(data[r].dtype)
        count.append(len(data[r]))
        missing.append(missing_vale_count[r])

    output_data = pd.DataFrame({
        'variable' : variables,
        'dtype' : dtypes,
        'number' : count,
        'missing data' : missing
    })

    return output_data

In [7]:
missing_column_processing = show_missing(new_data)
missing_column_processing

,variable,dtype,number,missing data
0,GarageType,object,1460,81


In [8]:
#fill in Garage relative with meaningfull value insted of leave it as NaN value for no Garage presentation

new_data['GarageType'] = new_data['GarageType'].fillna('NoG')

In [9]:
#plot to see whether a column was skewed or contain outlier
#notice that with randon forrest which is known as robust with skewed and outlier so there is not really need to deal with those kind of data so i wont deal with it
#most of the data is significant portion of the house so we should no deal with it
"""
n_rows = int(len(num_data_columns)/2)
n_cols = 3
fig, axs = plt.subplots(n_rows, n_cols, figsize=(20,20))
if(n_rows>1):
    axs = axs.flatten()
for i, column in enumerate(num_data_columns):
    ax = axs[i]
    ax.hist(raw_data[column], bins = 30)
    ax.set_title(column)
plt.tight_layout()
plt.show()
raw_data[num_data_columns].skew()
"""

'\nn_rows = int(len(num_data_columns)/2)\nn_cols = 3\nfig, axs = plt.subplots(n_rows, n_cols, figsize=(20,20))\nif(n_rows>1):\n    axs = axs.flatten()\nfor i, column in enumerate(num_data_columns):\n    ax = axs[i]\n    ax.hist(raw_data[column], bins = 30)\n    ax.set_title(column)\nplt.tight_layout()\nplt.show()\nraw_data[num_data_columns].skew()\n'

In [10]:
#pre-processing data

#min_max scaler
numerical_process = MinMaxScaler()
new_data[num_data_columns] = numerical_process.fit_transform(new_data[num_data_columns])

#label_encoding
label_process = LabelEncoder()
for column in label_data_columns:
    new_data[column] = label_process.fit_transform(new_data[column])

#one hot encoding
new_data = pd.get_dummies(new_data, columns=one_hot_columns)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(new_data,y,test_size=0.3,random_state=4)

In [12]:
sel = SelectFromModel(RandomForestRegressor(criterion='friedman_mse',n_estimators=1000))
sel.fit(X_train,y_train)

SelectFromModel(estimator=RandomForestRegressor(criterion='friedman_mse',
                                                n_estimators=1000))

In [13]:
selected_feat= new_data.columns[(sel.get_support())]
len(selected_feat) 

11

In [14]:
selected_feat

Index(['GrLivArea', 'TotalBsmtSF', '1stFlrSF', 'YearBuilt', 'GarageArea',
       'LotArea', 'BsmtFinSF1', 'TotRmsAbvGrd', 'OverallQual', 'FullBath',
       'GarageCars_3'],
      dtype='object')

In [20]:
after_choose_data = new_data[selected_feat]
X_train, X_test, y_train, y_test = train_test_split(after_choose_data, y, test_size=0.3,random_state=4)
rd_model = RandomForestRegressor(criterion='friedman_mse',n_estimators=1000)
rd_model.fit(X_train,y_train)
print(rd_model.score(X_test, y_test))


0.8841911520225183


In [24]:
predict = rd_model.predict(X_test)
MSError = mean_squared_error(y_test,predict)
print(np.sqrt(MSError))
print(y_test.mean())

24873.089266206734
178235.82420091325


In [16]:
import graphviz 

dot_data = tree.export_graphviz(dt, out_file=None, 
    feature_names=after_choose_data  
    class_names=new_raw_data['Exited'].unique().astype(str),  
    filled=True, rounded=True,  
    special_characters=True)
graph = graphviz.Source(dot_data)
graph

NameError: name 'dt' is not defined